In [1]:
import tensorflow as tf
import sys

In [68]:
image_path = '/home/datascience/projects/vsoft/data/test/Sausage/sau8.jpg'

In [69]:
# Read in the image_data
image_data = tf.gfile.FastGFile(image_path, 'rb').read()

In [64]:
# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("/tmp/labels.txt")]

In [65]:
# Read graph from file
with tf.gfile.FastGFile("/tmp/retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [70]:
with tf.Session() as sess:
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
    predictions = sess.run(softmax_tensor, \
             {'DecodeJpeg/contents:0': image_data})
    
    # Sort to show labels of first prediction in order of confidence
    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
    
    for node_id in top_k:
        human_string = label_lines[node_id]
        score = predictions[0][node_id]
        print('%s (score = %.5f)' % (human_string, score))
    

    filename = "results.txt"    
    with open(filename, 'a+') as f:
        f.write('\n**%s**\n' % (image_path))
        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
            f.write('%s (score = %.5f)\n' % (human_string, score))

Sausage (score = 0.78900)
Pepperoni (score = 0.21100)
